<a href="https://colab.research.google.com/github/karchx/spark/blob/main/spark-t.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset create CSV with pyspark

In [ ]:
!pip install pyspark pandas
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [2]:
import requests
import json

data = requests.get("https://www.defense.gov/data.json")
data = data.json()
with open("/tmp/ds_w.json", "w") as f:
  f.write(json.dumps(data["dataset"]))

In [3]:
# !cat /tmp/ds.json
!ls -l --block-size=M /tmp/ds_w.json

-rw-r--r-- 1 root root 1M Mar 13 04:16 /tmp/ds_w.json


In [4]:
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession

#conf = SparkConf().setMaster("local").setAppName("create_csv")

# Inicializo el Spark Context
spark = SparkSession.builder.master("local").appName("create_csv").getOrCreate()

#spark = SparkSession.builder.master("local").appName("create_csv").getOrCreate()
#spark = SQLContext(sc)

In [99]:
from pyspark.sql.functions import col, regexp_extract, expr, explode, first

df = spark.createDataFrame(data["dataset"])

df2 = df.select("*", explode(df.distribution))
df3 = df2.select("*", explode(df2.col))
pv = df3.select("*")\
  .groupBy([c for c in df3.columns if c not in {'contactPoint', 'distribution', 'publisher', 'col'}])\
  .pivot("key").agg(first("value"))

pv.drop("key", "value").show()


#pv.select("downloadURL", "mediaType", "title").write.csv("/tmp/ds.csv")

#df.withColumn("bureauCode", col("bureauCode").cast("string"))\
#.withColumn("contactPoint", col("contactPoint").cast("string"))\
#.write.csv("/tmp/ds.csv")
#df.select("distribution").show()

#df.select('publisher.name').show()

+------------+-----------+----------+-----------+--------------------+--------------------+--------------------+--------+--------------------+----------+-----------+-------+--------+--------------------+------------------+-----------+-----------------+---------+--------------------+--------------------+--------------------+
|       @type|accessLevel|bureauCode|dataQuality|         description|          identifier|             keyword|language|             license|  modified|programCode|spatial|temporal|               title|accrualPeriodicity|describedBy|            @type|accessURL|         downloadURL|           mediaType|               title|
+------------+-----------+----------+-----------+--------------------+--------------------+--------------------+--------+--------------------+----------+-----------+-------+--------+--------------------+------------------+-----------+-----------------+---------+--------------------+--------------------+--------------------+
|dcat:Dataset|     pub

In [83]:
from pyspark.sql.functions import size

df = spark.read.csv("/tmp/ds.csv", header=True)
df.show()
#df.select("publisher.name", "keyword").show()

+--------------------+----+----+
|                 _c0| _c1| _c2|
+--------------------+----+----+
|                NULL|NULL|NULL|
|                NULL|NULL|NULL|
|                NULL|NULL|NULL|
|                NULL|NULL|NULL|
|http://comptrolle...|NULL|NULL|
|http://comptrolle...|NULL|NULL|
|http://comptrolle...|NULL|NULL|
|http://comptrolle...|NULL|NULL|
|http://comptrolle...|NULL|NULL|
|http://comptrolle...|NULL|NULL|
|http://comptrolle...|NULL|NULL|
|http://comptrolle...|NULL|NULL|
|http://comptrolle...|NULL|NULL|
|http://comptrolle...|NULL|NULL|
|http://comptrolle...|NULL|NULL|
|http://comptrolle...|NULL|NULL|
|http://comptrolle...|NULL|NULL|
|http://comptrolle...|NULL|NULL|
|http://comptrolle...|NULL|NULL|
|http://comptrolle...|NULL|NULL|
+--------------------+----+----+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import size

df = spark.createDataFrame(data["dataset"])

df.select('publisher.name', 'keyword', size("keyword")).show()

In [80]:
!cat /tmp/ds.csv
# !rm /tmp/ds.csv

cat: /tmp/ds.csv: Is a directory


Goal

Create a silver layer of data that performs some transformations on the bronze data.

This data should provide two new tables:

providers
the unique name of each provider
keywords
how often each keyword occurred for each provider
Requirements
The following data should exist in the silver layer
providers
id: integer
name: string

keywords
id: integer
provider_id: string
keyword: string
count: integer

Any assumptions about the data, process, etc. are captured in the comments on this story
Any additional testing or capabilities are captured in the comments on this story
Blockers
This story requires that bronze data already exists